## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser
from datetime import datetime
from sys import getsizeof
import re

import operator
from scipy import stats

import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
init_notebook_mode(True)

#import seaborn as sns

from wand.image import Image as WImage

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo,py_profile

In [ ]:
plotdir = hpc_ext_path+'plots/enclosure_stability/Cold_Pond_01/'

In [ ]:
pkl_dir = hpc_ext_path+"datafiles/magnet_stability/pkl/"#enclosure_stability/"
pkl_file_slow = "enc_stability.run01.raw.slow.pkl"
#pkl_file_fast = "enc_stability.run01.raw.fast.pkl"
#pkl_file_fast_sparse = "hall_stability.run04.sparse.fast.pkl"

In [ ]:
# after data cleaning
df = pd.read_pickle(pkl_dir+pkl_file_slow)

In [ ]:
def pplot(fig):
    pio.write_image(fig, 'temp.pdf')
    img = WImage(filename='temp.pdf')
    return img

## Cleaning

In [ ]:
directory = "/home/ckampa/Dropbox/LogFiles/"#hpc_ext_path+'datafiles/magnet_stability/nmr/'
filename_slow = '2019-11-08 123301slow.txt' #'2019-08-23 162051slow.txt' # first file stopped Hall probe readings again
#filename_fast = '2019-09-04 080000.txt'

In [ ]:
df = pd.read_csv(directory+filename_slow)

In [ ]:
df.tail(1)

In [ ]:
df.columns = df.columns.str.lstrip(" ")

In [ ]:
#df.columns

In [ ]:
df.drop("", axis=1, inplace=True)

In [ ]:
dates = [parser.parse(row.Time) for row in df.itertuples()]# if isdatetime(row.Time.values[0])]

In [ ]:
df["Datetime"] = pd.to_datetime(dates)#.astype("str")
df = df.set_index("Datetime")

In [ ]:
#df['NMR [T]']

In [ ]:
df["NMR [T]"].tail()

In [ ]:
df = df[df["NMR [T]"] > 0.01]

In [ ]:
df["NMR [T]"].plot()

In [ ]:
nmr_change_per_sec = (df["NMR [T]"].diff() / (df.index.to_series().diff().dt.total_seconds()))#[1:]
nmr_change_per_min = (60*df["NMR [T]"].diff() / (df.index.to_series().diff().dt.total_seconds()))#[1:]
nmr_change_per_hour = (60**2*df["NMR [T]"].diff() / (df.index.to_series().diff().dt.total_seconds()))#[1:]

In [ ]:
#nmr_change_per_sec.plot()
#nmr_change_per_min.plot()
nmr_change_per_hour.plot()

In [ ]:
nmr_change_per_hour = (60**2*df["NMR [T]"].diff(40) / (df.index.to_series().diff(40).dt.total_seconds())) # 40 meas per hour

In [ ]:
nmr_change_per_hour.plot()

In [ ]:
gap_data = [
    [9.833887043189371, 2.289915966386555],
    [19.800664451827238, 2.0983193277310925],
    [29.966777408637874, 1.9067226890756301],
    [39.93355481727575, 1.750420168067227],
    [49.900332225913616, 1.6142857142857148],
    [69.83388704318936, 1.377310924369748],
    [99.93355481727573, 1.094957983193277],
    [149.96677740863785, 0.7873949579831936]
]

In [ ]:
gap_x = [p[0] for p in gap_data]
gap_y = [p[1] for p in gap_data]

In [ ]:
gap = pd.DataFrame({"x":gap_x, "y":gap_y})

In [ ]:
gap

In [ ]:
f = gap.iloc[6]
i = gap.iloc[5]

In [ ]:
slope = (f.y-i.y)/(f.x-i.x)

In [ ]:
slope / 100

~1 Gauss / 10 micron increase in gap size.

Linear expansion coeff. of pure iron is 12.0e-6 m / m (meters) K (or C)

So assuming L_0=1m (length of magnet), for a change of 4.5 deg C = 4.5 C (K) we should expect 54e-6 m = 54 micron increased length so the change in gap,
~54 microns, should decrease the field by ~5.4 Gauss. We actually see a shift of ~3 Gauss. This seems like a reasonable explanation.

Also, assuming L_0 ~ 1m (always, close for fixed gap),
**- ~1.2 Gauss / deg C**

From "plots/magnet_stability/2019-07-16/yoke_vs_nmr [t]_fit.pdf", we get **-1.118 Gauss / deg C in Yoke**

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
col = "Yoke"#"Roof"#"Yoke (center magnet)"
df_ = df#["11-12-19 09:20:00":"11-12-19 09:30:00"]
fig = make_subplots(specs=[[{"secondary_y": True}]])
#fig = scatter2d(df.index, df["NMR [T]"], show_plot=False)
fig.add_trace(
    go.Scatter(x=df_.index, y=df_["NMR [T]"], name="NMR", mode="lines"),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=df_.index, y=df_[col], name=col , mode="lines"),
    secondary_y=True
)

fig.update_xaxes(title_text="Datetime")

fig.update_yaxes(title_text="<b>NMR</b> |B| (T)", secondary_y=False)
fig.update_yaxes(title_text=f"<b>{col}</b> Temp (deg C)", secondary_y=True)

fig.show()

Seems that the change in temperature and Bfield effects occur at the same time (good!).

In [ ]:
for col in df.columns:
    fig = scatter2d(df.index, df[col], show_plot=False)
    fname = f"{plotdir}time/all/{col}_vs_time"
    pio.write_image(fig, fname+".pdf")
    pio.write_image(fig, fname+".png")

In [ ]:
#for col in df.columns:
#    if type(df[col].iloc[0]) is np.float64:
#        print(f"{col}: mean: {df[col].mean()}")

Looks like maybe probe 8F000... is in the field

In [ ]:
probes = np.array([])
for col in df.columns:
    #print(df_fast[col].dtype)
    if ("Raw" in col) or ("Cal" in col):#type(df[col].iloc[0]) is np.float64:
        #print(f"{col}")#": mean: {df[col].mean()}")
        probes = np.append(probes, col[:16])
probes = list(np.unique(probes))
print(probes)

### Slow Control Data Cleaning

In [ ]:
for probe in probes:
    for t in ["Cal", "Raw"]:
        df[f"{probe}_{t}_Bmag"] = (df[f"{probe}_{t}_X"]**2+df[f"{probe}_{t}_Y"]**2+df[f"{probe}_{t}_Z"]**2)**(1/2)
        df[f"{probe}_{t}_Res"] = df[f"{probe}_{t}_Bmag"] - df['NMR [T]']
        df[f"{probe}_{t}_Res_Rel"] = (df[f"{probe}_{t}_Bmag"] - df['NMR [T]']) / df['NMR [T]']

In [ ]:
for B in ["X", "Y", "Z"]:
    for probe in probes:
        for t in ["Cal", "Raw"]:
            #df[f"{probe}_{t}_Bmag"] = (df[f"{probe}_{t}_X"]**2+df[f"{probe}_{t}_Y"]**2+df[f"{probe}_{t}_Z"]**2)**(1/2)
            df[f"{probe}_{t}_{B}_Res"] = df[f"{probe}_{t}_{B}"] - df['NMR [T]']
            df[f"{probe}_{t}_{B}_Res_Rel"] = (df[f"{probe}_{t}_{B}"] - df['NMR [T]']) / df['NMR [T]']

In [ ]:
for probe in probes:
    for t in ["Cal", "Raw"]:
        fig = histo(df[f"{probe}_{t}_Bmag"], bins=50, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/{probe}_{t}_Bmag.pdf")
        pio.write_image(fig, plotdir+f"histo/{probe}_{t}_Bmag.png")

In [ ]:
for t in ["Cal", "Raw"]:
    for probe in probes:
        for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{B}_Res"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/residuals/component/{probe}_{t}_{B}_Res.pdf")
            pio.write_image(fig, plotdir+f"histo/residuals/component/{probe}_{t}_{B}_Res.png")

In [ ]:
for t in ["Cal", "Raw"]:
    for probe in probes:
        for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{B}_Res_Rel"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/relative_residuals/component/{probe}_{t}_{B}_Res_Rel.pdf")
            pio.write_image(fig, plotdir+f"histo/relative_residuals/component/{probe}_{t}_{B}_Res_Rel.png")

In [ ]:
for res in ["Res", "Res_Rel"]:
    if res == "Res":
        p = "residuals"
    else:
        p = "relative_residuals"
    for t in ["Cal", "Raw"]:
        for probe in probes:
            #for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{res}"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/{p}/{probe}_{t}_Bmag_{res}.pdf")
            pio.write_image(fig, plotdir+f"histo/{p}/{probe}_{t}_Bmag_{res}.png")

In [ ]:
df["seconds_delta"] = (df.index - df.index[0]).total_seconds()
df["hours_delta"] = (df.index - df.index[0]).total_seconds()/60**2
df["days_delta"] = (df.index - df.index[0]).total_seconds()/(24*60**2)

In [ ]:
type(df.days_delta) is not list

In [ ]:
# test new colors_list arg in scatter2d
fig = scatter2d(df.index, df["NMR [T]"], colors_list=df.days_delta, markers=True)

In [ ]:
pplot(fig)

In [ ]:
# cal

In [ ]:
for probe in probes:
    df[f"{probe}_Cal_Transverse"] = (df[f"{probe}_Cal_X"]**2+df[f"{probe}_Cal_Y"]**2)**(1/2)
    df[f"{probe}_Cal_Theta"] = np.arccos(df[f"{probe}_Cal_Z"]/df[f"{probe}_Cal_Bmag"])
    df[f"{probe}_Cal_Phi"] = np.arccos(df[f"{probe}_Cal_X"]/df[f"{probe}_Cal_Transverse"])

In [ ]:
for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = histo(df[f"{probe}_Cal_{angle}"], bins=30, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Cal_{angle}.pdf")
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Cal_{angle}.png")

In [ ]:
for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = scatter2d(df.index, df[f"{probe}_Cal_{angle}"], show_plot=False)
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Cal_{angle}.pdf")
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Cal_{angle}.png")

In [ ]:
# raw
for probe in probes:
    df[f"{probe}_Raw_Bmag"] = (df[f"{probe}_Raw_X"]**2+df[f"{probe}_Raw_Y"]**2+df[f"{probe}_Raw_Z"]**2)**(1/2)

for probe in probes:
    df[f"{probe}_Raw_Transverse"] = (df[f"{probe}_Raw_X"]**2+df[f"{probe}_Raw_Y"]**2)**(1/2)
    df[f"{probe}_Raw_Theta"] = np.arccos(df[f"{probe}_Raw_Z"]/df[f"{probe}_Raw_Bmag"])
    df[f"{probe}_Raw_Phi"] = np.arccos(df[f"{probe}_Raw_X"]/df[f"{probe}_Raw_Transverse"])

for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = histo(df[f"{probe}_Raw_{angle}"], bins=30, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Raw_{angle}.pdf")
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Raw_{angle}.png")

for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = scatter2d(df.index, df[f"{probe}_Raw_{angle}"], show_plot=False)
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Raw_{angle}.pdf")
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Raw_{angle}.png")

In [ ]:
df.to_pickle(pkl_dir+pkl_file_slow)

In [ ]:
getsizeof(df) / 1000/1000

In [ ]:
df.info()

## Plotting

### First Attempt

In [ ]:
len(df)

In [ ]:
len(df) * 90 / 60 / 60 / 24

Almost 10 days of data.

In [ ]:
#fig = scatter2d(df.index, df["NMR [T]"], show_plot=False);

In [ ]:
#pplot(fig)

In [ ]:
#fig = scatter2d(df.index, df["NMR [T]"])

In [ ]:
#plot(fig)

In [ ]:
fig = scatter2d(2*[df.index], [df["NMR [T]"], df[f"{probe}_Cal_Bmag"]], show_plot=False);

In [ ]:
plot(fig)

In [ ]:
#fig = scatter2d(df.index, df["Chamber Wall CH13"])

In [ ]:
fig = scatter2d(df.index, df[f"{probe}_Cal_Res_Rel"], show_plot=False);

In [ ]:
plot(fig)

~ -3e-5 change in residual after changing temperature.

In [ ]:
fig = histo(df[f"{probe}_Cal_Res_Rel"], bins=75, show_plot=False);
plot(fig)
#fig = histo(df[f"{probe}_Cal_Res_Rel"], bins=50, yscale='log')

In [ ]:
df["day"] = df.index.day

In [ ]:
fig = px.histogram(df, x=f"{probe}_Cal_Res_Rel", color="day")

In [ ]:
fig.update_layout(
    autosize=False,
    width=1800,
    height=800
    #margin=dict(l=20, r=20, t=20, b=20),
    #padding=dict(l=20, r=20, t=20, b=20),
    #paper_bgcolor="LightSteelBlue",
);

In [ ]:
#fig.show()

In [ ]:
plot(fig)

### look soon

In [ ]:
# look at below

In [ ]:
filename = plotdir+"NMR_vs_time"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
fig = scatter2d(df_fast[::10].index, df_fast[::10]["Cal_3_Bmag"], show_plot=False);
fig.layout.title = "|B_Hall]| [T] vs. Datetime";
fig.layout.yaxis.title = "|B_Hall]| [T]";

In [ ]:
filename = plotdir+"Hall_vs_time"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
P = 3
N = 60#10#30#10#20#10#30#60
df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]

In [ ]:
df_fast_5 = df_fast[N // 2::N]

In [ ]:
fig = scatter2d(df_fast_5.index, df_fast_5["Cal_3_Bmag_Avg"], markers=True, lines=False, size_list=3, show_plot=False);
fig.layout.title = "|B_Hall]| [T] vs. Datetime<br> 60 s Average";
fig.layout.yaxis.title = "|B_Hall]| [T]";
filename = plotdir+"Hall_vs_time_avg_60s"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
fig = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"], x_bins=len(df_fast) // 1000, show_plot=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Cal_3_Bmag"], show_plot=False);
fig.layout.title = "|B_Hall]| [T] vs. Datetime<br> 15 s Average";
fig.layout.yaxis.title = "|B_Hall]| [T]";
filename = plotdir+"Hall_vs_time_avg_15s"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
fig = scatter2d(df_slow.seconds_delta, df_slow[" NMR [T]"])

In [ ]:
fighall = scatter2d(df_fast[::100].seconds_delta, df_fast[::100]["Cal_3_Bmag"])

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"], 
                     xrange=[100000, 600000],
                     x_bins=5000, show_plot=True)

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"], x_bins=200, show_plot=True)

In [ ]:
fig.add_traces([fighall.data[0],figprof.data[0]])

In [ ]:
fig.layout.title = "Scatter, Profile: NMR and |B_hall| Comparison";
fig.layout.yaxis.title = "|B| [T]";
fig.layout.xaxis.title = "$\Delta t\ \\text{[s]}$";
filename = plotdir+"nmr_hall_comparison"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
#P = 3
#N = 1000#10#30#10#20#10#30#60
#df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
#df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]

In [ ]:
#df_fast_5 = df_fast[N // 2::N]

In [ ]:
#1000 / 60 /60 

In [ ]:
#figavg = scatter2d(df_fast_5.seconds_delta, df_fast_5["Cal_3_Bmag_Avg"], markers=True, lines=False, size_list=[5])

In [ ]:
#figavg.add_traces([figprof.data[0]])

In [ ]:
fig.add_traces([figprof.data[0]])

In [ ]:
P = 3
N = 60#10#30#10#20#10#30#60
df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]

In [ ]:
df_fast_5 = df_fast[N // 2::N]

In [ ]:
fig = scatter2d(df_fast_5["NMR"], df_fast_5["Cal_3_Bmag_Avg"], lines=False, markers=True)

In [ ]:
fig.layout.title = "|B_hall| (avg.) vs. NMR <br> 60 s Average";
fig.layout.yaxis.title = "|B_Hall| [T]";
fig.layout.xaxis.title = "NMR [T]";
filename = plotdir+"nmr_hall_correlation"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
df_fast.eval('Raw_3_Bmag = (Raw_3_Bx**2 + Raw_3_By**2 + Raw_3_Bz**2)**(1/2)',inplace=True)

In [ ]:
fig = histo(df_fast["Raw_3_Bmag"], bins=50)

In [ ]:
df_slow.columns

In [ ]:
# look at above

### continuing on

In [ ]:
temps_ch = [a for a in df.columns if ("CH" in a) and int(a[-2:])<21]# or "HVAC" in a]
temps_ch.append("Hall Element")
temps_ch.append("Floor")
temps_ch.append("Roof")
temps_ch.append("Yoke")
temps_ch.append("Yoke (center magnet)")

In [ ]:
#temps_ch

In [ ]:
#df[temps_ch].describe()

In [ ]:
#df[temps_ch].plot(kind="box")

In [ ]:
#fig = go.Figure()
#[fig.add_trace(go.Box(y=df[col], name=col)) for col in temps_ch]

#fig.show()

In [ ]:
fig = scatter2d([df.index for c in temps_ch],[df[c] for c in temps_ch],inline=True, show_plot=False);
fig.layout.title = "Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]";

In [ ]:
#fig.show()
#plot(fig)

In [ ]:
#fname = 'temps_scatter'
#fname = 'time/temps/temps_scatter_full'
#pio.write_image(fig, plotdir+fname+'.pdf')
#pio.write_image(fig, plotdir+fname+'.png')
#plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
probe = probes[2]

In [ ]:
fig.add_trace(go.Scatter(x=df.index, y=df[f"{probe}_Cal_T"], mode="lines", name=f"{probe}_Cal_T"));

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_full'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
plot(fig)

In [ ]:
temps_water = [col for col in df.columns if ("ICW" in col) or ("LCW" in col)]

In [ ]:
#temps_water

In [ ]:
fig = scatter2d([df.index for c in temps_water],[df[c] for c in temps_water],inline=True, show_plot=False);
fig.layout.title = "Water Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"

In [ ]:
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_water'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
temps_ambient = [col for col in df.columns if ("Ambient" in col)]
#temps_ambient

In [ ]:
fig = scatter2d([df.index for c in temps_ambient],[df[c] for c in temps_ambient],inline=True, show_plot=False);
fig.layout.title = "Ambient Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_ambient'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
temps_magnet = [col for col in df.columns if (("PS" in col) or ("Coil" in col) or ("Yoke" in col)) and ("Orange" not in col)]
#temps_magnet

In [ ]:
fig = scatter2d([df.index for c in temps_magnet],[df[c] for c in temps_magnet],inline=True, show_plot=False);
fig.layout.title = "Magnet Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_magnet'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
#df_ = df["11-09-19 00:00:00":"11-12-19 9:25:00"]
#fig = scatter2d(df_["Yoke (center magnet)"], df_["Floor"], colors_list=df.days_delta,)

In [ ]:
# correlation
fig = scatter2d([df["Yoke"], df["Yoke (center magnet)"]], 2*[df["NMR [T]"]], 
                colors_list=2*[df.hours_delta], #colorscale_list=["Viridis", "Aggrnyl"],
                lines=False, markers=True, legend_list=["Yoke", "Yoke (center magnet)"], show_plot=False,
                size_list=2*[5])
#fig = scatter2d(df["Yoke ()"], df["NMR [T]"], lines=False, markers=True,)

In [ ]:
#fname = 'temps_scatter'
fname = 'correlation_nmr_yokes'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
plot(fig)

In [ ]:
#fig.update_traces(marker=dict(size=5));
#plot(fig)

In [ ]:
# correlation
fig = scatter2d([df["Coil 1 (new)"], df["Coil 2 (old)"]], 2*[df["NMR [T]"]], 
                colors_list=2*[df.hours_delta], #colorscale_list=["Viridis", "Aggrnyl"],
                lines=False, markers=True, legend_list=["Coil 1 (new)", "Coil 2 (old)"], show_plot=False)
#fig = scatter2d(df["Yoke ()"], df["NMR [T]"], lines=False, markers=True,)

In [ ]:
#fname = 'temps_scatter'
fname = 'correlation_nmr_coils'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
#plot(fig)

In [ ]:
fig.update_traces(marker=dict(size=5));
plot(fig)

In [ ]:
# correlation
fig = scatter2d([df["LCW in Coil1"], df["LCW in Coil 2"]], 2*[df["NMR [T]"]], 
                colors_list=2*[df.hours_delta], #colorscale_list=["Viridis", "Aggrnyl"],
                lines=False, markers=True, legend_list=['LCW in Coil1', "LCW in Coil 2"], show_plot=False)
#fig = scatter2d(df["Yoke ()"], df["NMR [T]"], lines=False, markers=True,)

In [ ]:
#fname = 'temps_scatter'
fname = 'correlation_nmr_lcw_in_coils'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
plot(fig)

In [ ]:
fig.update_traces(marker=dict(size=5));
plot(fig)

In [ ]:
probe = probes[2]
probe

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_Bmag"], df.days_delta)

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_Bmag"] / df["NMR [T]"], df.days_delta)

In [ ]:
#corr_main = " LCW in barrel"
#corr_main = "Yoke"
corr_main = f"{probe}_Cal_T"
#corr_main = " Floor"

filestart = corr_main.strip().lower()

slope, intercept, r_value, p_value, std_err = stats.linregress(df[corr_main], df[f"{probe}_Raw_Bmag"]/df["NMR [T]"])

In [ ]:
slope

In [ ]:
intercept

In [ ]:
# rotation of axes

In [ ]:
y_old = df[f"{probe}_Raw_Bmag"]/df["NMR [T]"]

In [ ]:
theta_guess = np.arctan(slope)#np.arctan(1/slope)
theta_guess

In [ ]:
theta_guess/ np.pi

In [ ]:
#theta = 1.99*(np.pi/4)
theta = -theta_guess# * 1800

x = np.cos(theta) * df[corr_main] - np.sin(theta) * y_old
y = np.sin(theta) * df[corr_main] + np.cos(theta) * y_old

In [ ]:
fig = scatter2d(x, y, df.days_delta)

In [ ]:
slope_new, intercept_new, r_value_new, p_value_new, std_err_new = stats.linregress(x, y)

In [ ]:
slope_new, r_value_new

In [ ]:
slope, r_value

In [ ]:
y.name = "y_rot"

In [ ]:
fig = histo(y, bins=50)

In [ ]:
x.name = "x_rot"
fig = histo(x,bins=50)

In [ ]:
#y_old

In [ ]:
fig = histo(df[f"{probe}_Cal_T"], bins=50)

In [ ]:
y_old.name = "y_old"
fig = histo(y_old, bins=50)

In [ ]:
covar_rot = x.cov(y)
corr_rot = x.corr(y)

In [ ]:
covar_rot, corr_rot

In [ ]:
x = np.linspace(df[corr_main].min(), df[corr_main].max(), 500)
line = slope * x + intercept

In [ ]:
fig = scatter2d([df[corr_main], x],[df[f"{probe}_Raw_Bmag"]/df["NMR [T]"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
for B in ["X","Y","Z"]:
    fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_{B}"], df.days_delta)

In [ ]:
fig = scatter2d(df[f"NMR [T]"], df[f"{probe}_Raw_Bmag"], df[f"{probe}_Cal_T"], colorscale_list=["Viridis"])

In [ ]:
fig = scatter2d(df.index, df[f"{probe}_Cal_T"]-df["Hall Element"], inline=False)

In [ ]:
colors = ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
          'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
          'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
          'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
          'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
          'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
          'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
          'orrd', 'oryel', 'peach', 'phase', 'picnic', 'pinkyl', 'piyg',
          'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn', 'puor',
          'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu', 'rdgy',
          'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar', 'spectral',
          'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn', 'tealrose',
          'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'twilight',
          'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd']

In [ ]:
colors

In [ ]:
cs = sorted(list(px.colors.PLOTLY_SCALES.keys()))
[print(i, c) for i,c in enumerate(cs)];

In [ ]:
[15,3,8,12,4,5,0,9,10,]

In [ ]:
colors = ['Viridis', 'Cividis', 'Hot', 'Electric'] + cs

In [ ]:
colors

In [ ]:
px.colors.sequential.swatches()

In [ ]:
fname = 'correlations/nmr_vs_yoke_correlation'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')

In [ ]:
fig = scatter2d(df["Yoke (center magnet)"], df["Magnet Current [A]"], lines=False, markers=True,)

In [ ]:
df_cur = df[df["Magnet Current [A]"] > 50]#df[df["Yoke (center magnet)"] > 22]

In [ ]:
fig = scatter2d(df_cur["Yoke (center magnet)"], df_cur["Magnet Current [A]"], lines=False, markers=True,)

In [ ]:
#df_cur = df[df["Yoke (center magnet)"] > 22]

In [ ]:
fig = scatter2d(df_cur["Yoke (center magnet)"], df_cur["NMR [T]"], lines=False, markers=True,)

In [ ]:
fig = scatter2d(df["Magnet Current [A]"], df["NMR [T]"], lines=False, markers=True,)
fig = scatter2d(df_cur["Magnet Current [A]"], df_cur["NMR [T]"], lines=False, markers=True, size_list=[5])

In [ ]:
#df_ = df["11-14-19 00:00:00":]#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
#df_ = df["11-13-19 02:00:00":"11-13-19 16:00:00"]
df_ = df
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
#corr_main = " LCW in barrel"
#corr_main = "Yoke"
#corr_main = '8F0000000D5B373A_Cal_Bmag'
#corr_main = '140000000D5FB53A_Cal_T'
corr_main = "Yoke (center magnet)"
#corr_main = "Floor"

filestart = corr_main.strip().lower()

slope, intercept, r_value, p_value, std_err = stats.linregress(df_[corr_main], df_["NMR [T]"])

In [ ]:
slope

In [ ]:
line = slope * df_[corr_main] + intercept

In [ ]:
fig = scatter2d(2*[df_[corr_main]],[df_["NMR [T]"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit.png")

In [ ]:
# rotation of axes

In [ ]:
y_old = df["NMR [T]"]

In [ ]:
theta_guess = np.arctan(slope)#np.arctan(1/slope)
theta_guess

In [ ]:
theta_guess/ np.pi

In [ ]:
#theta = 1.99*(np.pi/4)
theta = -theta_guess #* 0.95# * 1800

x = np.cos(theta) * df[corr_main] - np.sin(theta) * y_old
y = np.sin(theta) * df[corr_main] + np.cos(theta) * y_old

In [ ]:
fig = scatter2d(x, y, df.days_delta)

In [ ]:
df["NMR_const_rot"] = y

In [ ]:
df_ = df[:"11-11-19 12:00:00"]#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_const_rot"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

[print(s) for s in sorted_primary_corr if ("Yoke" in s[0])];
sorted_primary_corr

In [ ]:
df_ = df#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_const_rot"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

#sorted_primary_corr
[print(s) for s in sorted_primary_corr if ("Yoke" in s[0])];

In [ ]:
slope_new, intercept_new, r_value_new, p_value_new, std_err_new = stats.linregress(x, y)

In [ ]:
slope_new, r_value_new

In [ ]:
slope, r_value

In [ ]:
y.name = "y_rot"

In [ ]:
fig = histo(y, bins=50)

In [ ]:
x.name = "x_rot"
fig = histo(x,bins=50)

In [ ]:
#y_old

In [ ]:
fig = histo(df[f"{probe}_Cal_T"], bins=50)

In [ ]:
y_old.name = "y_old"
fig = histo(y_old, bins=50)

In [ ]:
covar_rot = x.cov(y)
corr_rot = x.corr(y)

In [ ]:
covar_rot, corr_rot

In [ ]:
x = np.linspace(df[corr_main].min(), df[corr_main].max(), 500)
line = slope * x + intercept

In [ ]:
fig = scatter2d([df[corr_main], x],[df[f"{probe}_Raw_Bmag"]/df["NMR [T]"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
df["NMR_const"] = df["NMR [T]"] - slope * df[corr_main] - intercept + df["NMR [T]"].mean()

In [ ]:
fig = scatter2d(2*[df[corr_main]],[df["NMR_const"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.png")

In [ ]:
fig = histo(df["NMR [T]"],bins=50,inline=True);
pio.write_image(fig,plotdir+f"hist_NMR.pdf")
pio.write_image(fig,plotdir+f"hist_NMR.png")
fig = histo(df["NMR_const"],bins=50,inline=True);
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.pdf")
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.png")

In [ ]:
df["Zaber_Meas_Encoder_Z"].plot()

In [ ]:
df["Zaber_Meas_MicroStep_Z"].plot()

In [ ]:
#df["8F0000000D5B373A_Cal_Transverse"].plot()
df["8F0000000D5B373A_Cal_Theta"].plot()

In [ ]:
df["NMR [T]"].plot()

In [ ]:
df["Magnet Current [A]"].plot()

In [ ]:
fig = scatter2d(df.index,df["Magnet Current [A]"], inline=True)

In [ ]:
fig = scatter2d(df.index,df["NMR [T]"], inline=True)

In [ ]:
fig = scatter2d(df.index,df["Zaber_Meas_MicroStep_Z"], inline=True)

In [ ]:
df_ = df#[:"11-11-19 12:00:00"]#:"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_const_rot"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

#[print(s) for s in sorted_primary_corr if ("Yoke" in s[0])];
sorted_primary_corr

In [ ]:
#corr_main = "LCW in Coil 2"
#corr_main = "LCW from PS"
#corr_main = 'Zaber_Meas_MicroStep_Z'
#corr_main = '140000000D5FB53A_Cal_T'
#corr_main = "Yoke"
#corr_main = '140000000D5FB53A_Cal_Res'
#corr_main = "Yoke (center magnet)"
#corr_main = "Floor"
#corr_main = "Roof"
corr_main = "Chamber Wall CH16"

filestart = corr_main.strip().lower()

slope, intercept, r_value, p_value, std_err = stats.linregress(df_[corr_main], df_["NMR_const_rot"])

In [ ]:
slope

In [ ]:
line = slope * df_[corr_main] + intercept

In [ ]:
fig = scatter2d(2*[df_[corr_main]],[df_["NMR_const_rot"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
# rotation of axes

In [ ]:
y_old = df["NMR_const_rot"]

In [ ]:
theta_guess = np.arctan(slope)#np.arctan(1/slope)
theta_guess

In [ ]:
theta_guess/ np.pi

In [ ]:
#theta = 1.99*(np.pi/4)
theta = -theta_guess#*0.6# * 1800

x = np.cos(theta) * df[corr_main] - np.sin(theta) * y_old
y = np.sin(theta) * df[corr_main] + np.cos(theta) * y_old

In [ ]:
fig = scatter2d(x, y, df.days_delta)

In [ ]:
df["NMR_const_rot2"] = y

In [ ]:
fig = histo([df[col] for col in ["NMR [T]", "NMR_const_rot", "NMR_const_rot2"]], bins=75)

In [ ]:
df_ = df#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_const_rot2"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
#corr_main = 'LCW in Coil 2'
corr_main = "Coil 1 (new)"
#corr_main = "Yoke"
#corr_main = '140000000D5FB53A_Cal_Res'
#corr_main = "Yoke (center magnet)"
#corr_main = "Floor"

filestart = corr_main.strip().lower()

slope, intercept, r_value, p_value, std_err = stats.linregress(df_[corr_main], df_["NMR_const_rot2"])

In [ ]:
slope

In [ ]:
line = slope * df_[corr_main] + intercept

In [ ]:
fig = scatter2d(2*[df_[corr_main]],[df_["NMR_const_rot2"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
# rotation of axes

In [ ]:
y_old = df["NMR_const_rot2"]

In [ ]:
theta_guess = np.arctan(slope)#np.arctan(1/slope)
theta_guess

In [ ]:
theta_guess/ np.pi

In [ ]:
#theta = 1.99*(np.pi/4)
theta = -theta_guess# * 1800

In [ ]:
x = np.cos(theta) * df[corr_main] - np.sin(theta) * y_old
y = np.sin(theta) * df[corr_main] + np.cos(theta) * y_old

In [ ]:
fig = scatter2d(x, y, df.days_delta, ["Viridis"])

In [ ]:
df["NMR_const_rot3"] = y

In [ ]:
fig = histo([df[col] for col in ["NMR [T]", "NMR_const_rot", "NMR_const_rot2", "NMR_const_rot3"]], bins=75)

In [ ]:
fig = scatter2d(df.index, df["NMR [T]"])
fig = scatter2d(df.index, df["NMR_const_rot"])
fig = scatter2d(df.index, df["NMR_const_rot2"])
fig = scatter2d(df.index, df["NMR_const_rot3"])

In [ ]:
fig = scatter2d(df["Zaber_Meas_MicroStep_Z"], df["NMR_const_rot3"], df["days_delta"], ["Viridis"])

## NMR Equation / Fitting

In [ ]:
df_ = df#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
df["Zaber_Meas_MicroStep_Z"].plot()

In [ ]:
import scipy.optimize as optimization

In [ ]:
def field_strength_func(DATA, A, B, C, D):
    YOKE, ENC, WATER = DATA
    return A * YOKE + B * ENC + C * WATER + D

In [ ]:
guess = np.array([-1e-4, -1e-5, -1e-6, 1.249])

In [ ]:
#x_dat = np.vstack([df["Yoke (center magnet)"],df["Chamber Wall CH16"],df["Coil 1 (new)"]])
x_dat = np.vstack([df["Yoke"],df["Floor"],df["ICW HE1 out"]])
#x_dat2 = np.vstack([df["Yoke (center magnet)"],df["140000000D5FB53A_Cal_T"],df["LCW in Coil 2"]])
x_dat2 = np.vstack([df["Yoke (center magnet)"],df["140000000D5FB53A_Cal_T"],df["LCW in Coil 2"]])

In [ ]:
x_dat[2]

In [ ]:
field_strength_func(x_dat, -1e-4, -1e-5, -1e-6, 1.249)

In [ ]:
params, cov = optimization.curve_fit(field_strength_func, x_dat, df["NMR [T]"], guess)
params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df["NMR [T]"], guess)

In [ ]:
A_fit, B_fit, C_fit, D_fit = params
A_fit2, B_fit2, C_fit2, D_fit2 = params2

In [ ]:
params, params2

In [ ]:
cov, cov2

In [ ]:
df["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
df["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)

In [ ]:
fig = scatter2d([df["NMR_lsq"], df["NMR_lsq2"]], 2*[df["NMR [T]"]], 2*[df.days_delta], show_plot=False)
plot(fig)

In [ ]:
fig = scatter2d(df["NMR_lsq"], df["NMR [T]"], df.days_delta, ["Viridis"], show_plot=False)
#fig = scatter2d(df["NMR_lsq2"], df["NMR [T]"], df.days_delta, ["Viridis"], show_plot=False)
fig.update_xaxes(title="|B| (calculated)")
fig.update_yaxes(title="NMR [T] (measured)")
fig.update_layout(title="Measured vs. Calculated |B|")
plot(fig)

In [ ]:
filename = "NMR_measured_vs_calculated"
pio.write_image(fig, plotdir+filename+".pdf")
pio.write_image(fig, plotdir+filename+".png")

In [ ]:
df["LSQ_RES_REL"] = (df["NMR [T]"] - df["NMR_lsq"]) / df["NMR [T]"]
df["LSQ2_RES_REL"] = (df["NMR [T]"] - df["NMR_lsq2"]) / df["NMR [T]"]
df["LSQ_RES"] = (df["NMR [T]"] - df["NMR_lsq"])
df["LSQ2_RES"] = (df["NMR [T]"] - df["NMR_lsq2"])

In [ ]:
fig = scatter2d(df["NMR_lsq"], df["LSQ_RES_REL"], df.days_delta, ["Viridis"], show_plot=False)
#fig = scatter2d(df["NMR_lsq2"], df["LSQ2_RES_REL"], df.days_delta, ["Viridis"], show_plot=False)
plot(fig)

In [ ]:
fig.update_traces(marker=dict(size=5))
plot(fig)

In [ ]:
fig = scatter2d(df.index, df["LSQ2_RES_REL"], df.days_delta, ["Viridis"], show_plot=False)
plot(fig)

In [ ]:
fig.update_traces(marker=dict(size=5));
plot(fig)

In [ ]:
fig = histo(df["LSQ2_RES"], bins=100, show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) )")
fig.update_layout(title="|B| Calculation Residuals")
plot(fig)

In [ ]:
fig = histo(df["LSQ2_RES_REL"], bins=100, yscale="log", show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) ) / NMR_Meas")
fig.update_layout(title="|B| Calculation Relative Residuals");
plot(fig)

In [ ]:
filename = "NMR_calculation_res_hist"
pio.write_image(fig, plotdir+filename+".pdf")
pio.write_image(fig, plotdir+filename+".png")

In [ ]:
fig = histo([df["LSQ_RES_REL"],df["LSQ2_RES_REL"],], bins=75, opacity=0.7)

In [ ]:
import scipy.stats as stats

In [ ]:
chi2 = np.sum(np.square((df["NMR [T]"] - df["NMR_lsq2"])/2.9e-6))

In [ ]:
chi2

In [ ]:
dof = len(df) - 4 # 4 param fit
chis = np.linspace(0, 10000, 100000)
pdf = stats.chi2.pdf(chis, dof)

In [ ]:
dof

In [ ]:
chi2 / dof

In [ ]:
plt.plot(chis, pdf, label=r"p.d.f., $\nu = 2$")

In [ ]:
dof

plt.plot(chis/dof, pdf, label=r"p.d.f., $\nu = 2$")
plt.plot([chi2/dof, chi2/dof],[0, (pdf).max()], "r--", label= rf"$\chi^2_{{min}} =${chi2/dof:.3f}")
plt.xlabel(r"$\chi^2$")
plt.ylabel("p.d.f.")
plt.legend();

### Checking out current (when we have DMM)

In [ ]:
df_ = df[df["Magnet Current [A]"] > 0]

In [ ]:
len(df_)

In [ ]:
len(df)

In [ ]:
df_["NMR [T]"].plot()

In [ ]:
fig = scatter2d(df_["Magnet Current [A]"], df_["NMR [T]"], df_.days_delta)

### More Fitting (using current)

In [ ]:
df[df["Magnet Current [A]"] != 0].tail()

In [ ]:
df_ = df[:"11-11-19 02:55:00"]#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
import scipy.optimize as optimization

In [ ]:
def field_strength_func(DATA, A, B, C, D):
    CURRENT, YOKE, ENC, WATER = DATA
    return A * YOKE + B * ENC + C * WATER + D * CURRENT

In [ ]:
guess = np.array([-1e-4, -1e-5, -1e-6, 1.249 / df_["Magnet Current [A]"].max()])

In [ ]:
x_dat = np.vstack([df_["Magnet Current [A]"],df_["Yoke (center magnet)"],df_["Chamber Wall CH16"],df_["Coil 1 (new)"]])
x_dat2 = np.vstack([df_["Magnet Current [A]"],df_["Yoke (center magnet)"],df_["140000000D5FB53A_Cal_T"],df_["LCW in Coil 2"]])

In [ ]:
x_dat[2]

In [ ]:
field_strength_func(x_dat, -1e-4, -1e-5, -1e-6, 1.249 / 230)

In [ ]:
params, cov = optimization.curve_fit(field_strength_func, x_dat, df_["NMR [T]"], guess)
params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df_["NMR [T]"], guess)

In [ ]:
A_fit, B_fit, C_fit, D_fit = params
A_fit2, B_fit2, C_fit2, D_fit2 = params2

In [ ]:
params, params2

In [ ]:
cov, cov2

In [ ]:
df_["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
df_["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)

In [ ]:
fig = scatter2d([df_["NMR_lsq"], df_["NMR_lsq2"]], 2*[df_["NMR [T]"]], 2*[df_.days_delta])

In [ ]:
fig = scatter2d(df_["NMR_lsq"], df_["NMR [T]"], df_.days_delta, ["Viridis"], show_plot=False)
fig.update_xaxes(title="|B| (calculated)")
fig.update_yaxes(title="NMR [T] (measured)")
fig.update_layout(title="Measured vs. Calculated |B|")

In [ ]:
#filename = "NMR_measured_vs_calculated"
#pio.write_image(fig, plotdir+filename+".pdf")
#pio.write_image(fig, plotdir+filename+".png")

In [ ]:
df_["LSQ_RES_REL"] = (df_["NMR [T]"] - df_["NMR_lsq"]) / df_["NMR [T]"]
df_["LSQ2_RES_REL"] = (df_["NMR [T]"] - df_["NMR_lsq2"]) / df_["NMR [T]"]
df_["LSQ_RES"] = (df_["NMR [T]"] - df_["NMR_lsq"])
df_["LSQ2_RES"] = (df_["NMR [T]"] - df_["NMR_lsq2"])

In [ ]:
fig = scatter2d(df_["NMR_lsq2"], df_["LSQ2_RES_REL"], df_.days_delta, ["Viridis"], show_plot=True)

In [ ]:
fig.update_traces(marker=dict(size=5))

In [ ]:
fig = scatter2d(df_.index, df_["LSQ_RES_REL"], df_.days_delta, ["Viridis"], show_plot=True)

In [ ]:
fig.update_traces(marker=dict(size=5))

In [ ]:
fig = histo(df_["LSQ2_RES"], bins=50, show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) )")
fig.update_layout(title="|B| Calculation Residuals")

In [ ]:
fig = histo(df_["LSQ_RES_REL"], bins=50, show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) ) / NMR_Meas")
fig.update_layout(title="|B| Calculation Relative Residuals")

In [ ]:
#filename = "NMR_calculation_res_hist"
#pio.write_image(fig, plotdir+filename+".pdf")
#pio.write_image(fig, plotdir+filename+".png")

In [ ]:
fig = histo([df_["LSQ_RES_REL"],df_["LSQ2_RES_REL"],], bins=75, opacity=0.7)

## Look here sometime

In [ ]:
df_ = df#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_const_rot3"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
corr_main = 'LCW in Coil 2'
#corr_main = "Yoke"
#corr_main = '140000000D5FB53A_Cal_Res'
#corr_main = "Yoke (center magnet)"
#corr_main = "Floor"

filestart = corr_main.strip().lower()

slope, intercept, r_value, p_value, std_err = stats.linregress(df_[corr_main], df_["NMR_const_rot2"])

In [ ]:
slope

In [ ]:
line = slope * df_[corr_main] + intercept

In [ ]:
fig = scatter2d(2*[df_[corr_main]],[df_["NMR_const_rot2"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
# rotation of axes

In [ ]:
y_old = df["NMR_const_rot2"]

In [ ]:
theta_guess = np.arctan(slope)#np.arctan(1/slope)
theta_guess

In [ ]:
theta_guess/ np.pi

In [ ]:
#theta = 1.99*(np.pi/4)
theta = -theta_guess# * 1800

In [ ]:
x = np.cos(theta) * df[corr_main] - np.sin(theta) * y_old
y = np.sin(theta) * df[corr_main] + np.cos(theta) * y_old

In [ ]:
fig = scatter2d(x, y, df.days_delta)

In [ ]:
df["NMR_const_rot3"] = y

In [ ]:
fig = histo([df[col] for col in ["NMR [T]", "NMR_const_rot", "NMR_const_rot2", "NMR_const_rot3"]], bins=75)

In [ ]:
fig = scatter2d(df.index, df["Magnet Current [A]"])

In [ ]:
fig = scatter2d(df.index, df["Desk Ambient"])

In [ ]:
fig = scatter2d(2*[df.index], [(df[col]-df[col].mean())/df[col].std() for col in ["Magnet Current [A]", "Desk Ambient"]])

In [ ]:
df_per = df["11-09-2019 12:00:00":]

In [ ]:
fig = scatter2d(2*[df_per.index], [(df_per[col]-df_per[col].mean())/df_per[col].std() for col in ["Magnet Current [A]", "Desk Ambient"]])

In [ ]:
np.fft.fft(df_per["Magnet Current [A]"])

In [ ]:
import scipy.fftpack

In [ ]:
from scipy.signal import blackman

In [ ]:
elapsed_ = (df_per.index[-1] - df_per.index[0])
elapsed = elapsed_.total_seconds()
elapsed_, elapsed

In [ ]:
N = len(df_per) # samples
T = N / elapsed # sample per second (sampling freq)

In [ ]:
T

In [ ]:
1 / T

In [ ]:
df_per["seconds_delta"] = (df_per.index - df_per.index[0]).total_seconds()

In [ ]:
df_per["seconds_delta"].head()

In [ ]:
w = blackman(N)

In [ ]:
x = df_per["seconds_delta"].values
y = df_per["Magnet Current [A]"].values
yf = scipy.fftpack.fft(y*w)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

In [ ]:
N

In [ ]:
fig = scatter2d(xf, 2.0/N * np.abs(yf[:N//2]))

In [ ]:
# try 2

In [ ]:
y = y - y.mean()

In [ ]:
sig_fft = scipy.fftpack.fft(y)
power = np.abs(sig_fft)
sample_freq = scipy.fftpack.fftfreq(y.size, d=9.)

In [ ]:
sample_freq

In [ ]:
y.size

In [ ]:
fig = scatter2d(x, y)

In [ ]:
sample_freq

In [ ]:
df_ = pd.DataFrame({"f":sample_freq, "p":power})#, columns=["f", "p"])
df_.sort_values(by="f", inplace=True)

In [ ]:
fig = scatter2d(df_.f, df_.p)

In [ ]:
fig = scatter2d(sample_freq, power)

In [ ]:
# 6 hour period
f_exp = 1 / (6*60*60)
f_exp

In [ ]:
df_per.head(1)

In [ ]:
df_per.tail(1)

In [ ]:
elapsed_ = (df_per.index[-1] - df_per.index[0])
elapsed = elapsed_.total_seconds()
elapsed_, elapsed

In [ ]:
44001 / 60 / 60

In [ ]:
N

In [ ]:
elapsed / 60

In [ ]:
df_cur.head()

In [ ]:
df.head()

In [ ]:
fig = scatter2d(df_fast.iloc[299:902].seconds_delta, df_fast.iloc[299:902]["Cal_3_Bmag"], show_plot=False)

In [ ]:
figprof_ = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"],
                      xrange=[300.11895,900.35695],
                      x_bins=20, show_plot=True)

In [ ]:
figprof = py_profile(df_fast.iloc[300:901].seconds_delta, df_fast.iloc[300:901]["Cal_3_Bmag"],
                     x_bins=20, show_plot=False)

In [ ]:
figprof_.add_traces([figprof.data[0]])

In [ ]:
fig.add_traces([figprof.data[0]])

In [ ]:
15.00595*2

In [ ]:
315.1249-15.00595

In [ ]:
df_fast.iloc[300:901].seconds_delta.describe()

In [ ]:
# bin size
345.1368 - 315.1249

In [ ]:
(end - start) / bins

In [ ]:
start = 300.11895
end = 900.35695

delta_t = end - start

bins = 20
step = (end - start) / bins

average_time = np.zeros(bins)
average_cal = np.zeros(bins)
std_cal = np.zeros(bins)

In [ ]:
for i in range(bins):
    minx = start + i*step
    maxx = minx + step
    midx = minx + step / 2
    cal = df_fast.query(f"(seconds_delta >= {minx}) & (seconds_delta < {maxx})")["Cal_3_Bmag"]
    average_cal[i] = cal.mean()#cal.sum() / step
    std_cal[i] = cal.std()
    average_time[i] = midx

In [ ]:
figavg = scatter2d(average_time, average_cal, lines=False, markers=True, size_list=[4])

In [ ]:
fig.add_traces([figavg.data[0]])

In [ ]:
(30**(1/2))

In [ ]:
figprof, tprof = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"],
                     xrange=[300,900],
                     x_bins=20, show_plot=False, return_tprof=True)

In [ ]:
fig.add_traces([figprof.data[0]])

In [ ]:
df_nmr = df_slow.query("(seconds_delta > 299) & (seconds_delta < 901)")

In [ ]:
fignmr = scatter2d(df_nmr.seconds_delta, df_nmr[" NMR [T]"], show_plot=False)

In [ ]:
fig.add_traces([fignmr.data[0]])

In [ ]:
tprof

In [ ]:
tprof.GetBinEntries(0)

In [ ]:
# test speed

In [ ]:
%timeit figprof = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"],xrange=[300,900],x_bins=20, show_plot=False)

In [ ]:
def pythonic_profile(x, y, x_bins, xrange=None, show_plot=True):
    if xrange == None:
        xrange = [x.min(), x.max()]
    start = xrange[0]
    end = xrange[1]

    delta_t = end - start

    bins = x_bins
    step = (end - start) / bins

    average_time = np.zeros(bins)
    average_cal = np.zeros(bins)
    std_cal = np.zeros(bins)

    for i in range(bins):
        minx = start + i*step
        maxx = minx + step
        midx = minx + step / 2
        print(minx, midx, maxx)
        #cal = x.query(f"(seconds_delta >= {minx}) & (seconds_delta < {maxx})")#["Cal_3_Bmag"]
        xmask = (x >= minx) & (x <= maxx)
        cal = y[xmask]
        print(i, len(cal))
        average_cal[i] = cal.mean()#cal.sum() / step
        std_cal[i] = cal.std() / len(cal)
        average_time[i] = midx

    fig = scatter2d(average_time, average_cal, lines=False, markers=True, size_list=[4], show_plot=False)
    figavg.update_traces(error_y = dict(type="data", array=std_cal, visible=True))
    figavg.update_traces(error_x = dict(type="constant", value=step/2, visible=True))
    
    return fig

In [ ]:
figprof = pythonic_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"], x_bins=20, 
                           xrange=[300,900], show_plot=True)

In [ ]:
df_fast.Cal_3_Bmag.describe()

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast["Cal_3_Bmag"],
                     xrange=[300,900],
                     x_bins=20, show_plot=True)

In [ ]:
start = 300
end = 900

delta_t = end - start

bins = 20
step = (end - start) / bins

average_time = np.zeros(bins)
average_cal = np.zeros(bins)
std_cal = np.zeros(bins)

In [ ]:
for i in range(bins):
    minx = start + i*step
    maxx = minx + step
    midx = minx + step / 2
    cal = df_fast.query(f"(seconds_delta >= {minx}) & (seconds_delta < {maxx})")["Cal_3_Bmag"]
    average_cal[i] = cal.mean()#cal.sum() / step
    std_cal[i] = cal.std() / np.sqrt(len(cal))
    average_time[i] = midx

In [ ]:
figavg = scatter2d(average_time, average_cal, lines=False, markers=True, size_list=[4])

In [ ]:
#figavg.update_traces(error_y = dict(type="data", array=std_cal, visible=True))
#figavg.update_traces(error_x = dict(type="constant", value=step/2, visible=True))

In [ ]:
figprof.add_traces([figavg.data[0]])

In [ ]:
P = 3
N = 30#60#10#30#10#20#10#30#60
df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]

df_fast_5 = df_fast[N // 2::N]

In [ ]:
figavgr = scatter2d(df_fast[300:901].seconds_delta, df_fast[300:901].Cal_3_Bmag_Avg, lines=False, markers=True, size_list=[4])

In [ ]:
figprof.add_traces([figavgr.data[0]])

In [ ]:
figavgr2 = scatter2d(df_fast_5[10:30].seconds_delta, df_fast_5[10:30].Cal_3_Bmag_Avg, lines=False, markers=True, size_list=[4])

In [ ]:
figprof.add_traces([figavgr2.data[0]])

In [ ]:
avg_cal3 = [15+(i* 30) for i in range(20)]

In [ ]:
df_avg = df[]

In [ ]:
figavg = 

In [ ]:
fig.layout.yaxis.title = "|Hall| [T]"

In [ ]:
iplot(fig)

In [ ]:
fig = scatter2d(df_fast[::10].index, df_fast[::10]["Cal_3_Bmag"])

In [ ]:
fig = scatter2d(df_fast[::100].NMR, df_fast[::100].Cal_3_Bmag, lines=False, markers=True)

In [ ]:
#for cal in ["1", "2", "3", "4", "5"]:
#    fig = scatter2d(df_fast.index, df_fast[f"Cal_{cal}_Bmag"])

Probe #4 is in the magnet.

In [ ]:
P = 3
N = 10#30#10#20#10#30#60
df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]

In [ ]:
df_fast_5 = df_fast[N // 2::N]

In [ ]:
#df_fast_5.loc[:, "Cal_Mag_Avg"] = [df_fast.iloc[i- for i in range(len(df_fast_5))]

In [ ]:
### 60 second

In [ ]:
fig = px.scatter_matrix(df_fast[::100], dimensions=["NMR","Cal_1_Bmag","Cal_2_Bmag","Cal_3_Bmag",])
fig.show()

In [ ]:
len(df_fast)

In [ ]:
#perc = 0.001
#bad_map = (df_fast[f"Cal_{P}_Bmag"] < df_fast[f"Cal_{P}_Bmag"].quantile(perc)) | (df_fast[f"Cal_{P}_Bmag"] > df_fast[f"Cal_{P}_Bmag"].quantile(1-perc))
#df_fast[bad_map]

In [ ]:
#df_fast2 = df_fast[~bad_map]

In [ ]:
#df_fast2.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast2[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
#df_fast2.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast2["NMR"] - df_fast2[f"Cal_{P}_Bmag_Avg"])/df_fast2["NMR"]

In [ ]:
#df_fast2_5 = df_fast2[N // 2::N]

In [ ]:
#fig = px.scatter_matrix(df_fast2, dimensions=["NMR","Cal_1_Bmag","Cal_2_Bmag","Cal_3_Bmag","Cal_4_Bmag","Cal_5_Bmag",])
#fig.show()

In [ ]:
#plot(fig)

In [ ]:
#fig = scatter2d(df_fast2.NMR, df_fast2[f"Cal_{P}_Bmag"], lines=False, markers=True)

In [ ]:
from scipy import stats

In [ ]:
primary_corr = dict()
for col in df_slow.columns:
    if (df_slow[col].dtype == np.float64):
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_slow[col], df_slow[" NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
primary_corr = dict()
for col in df_fast.columns:
    if (df_fast[col].dtype == np.float64) & ("T" not in col): #& ("Cal_4" in col):
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_fast[col], df_fast["NMR"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

In [ ]:
df_fast["Cal_3_Bmag_Avg_Res_Rel"].describe()

In [ ]:
df_fast["NMR"].describe()

In [ ]:
len(df_fast)

In [ ]:
df_fast[f"Cal_{P}_Res_Rel"].describe()

In [ ]:
fig = histo(df_fast[f"Cal_{P}_Res"], bins=100, yscale='linear', cut=[0.,0.999])

In [ ]:
fig = histo(df_fast[f"Cal_{P}_Res_Rel"], bins=100, yscale='linear', cut=[0.,0.999])

In [ ]:
df_fast[df_fast[f"Cal_{P}_Res_Rel"] == df_fast[f"Cal_{P}_Res_Rel"].max()]

In [ ]:
fig = histo(df_fast[f"Cal_{P}_Res_Rel"], bins=100, yscale='linear', cut=[0.,0.999], show_plot=False)

In [ ]:
fig.layout.xaxis.title = "(NMR [T] - |Hall| [T]) / (NMR [T])"
fig.layout.title = "Histo: Relative Residuals"

In [ ]:
filename = plotdir+"rel_residuals"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
fig = histo(df_fast[f"Cal_{P}_Bmag_Avg_Res_Rel"], bins=100, yscale='linear', cut=[0.,0.999], show_plot=True)

In [ ]:
fig = scatter2d([df_fast.index,df_fast.index,df_fast_5.index], [df_fast.Cal_1_Bmag,df_fast.Cal_1_Bmag_Avg,df_fast_5.Cal_1_Bmag_Avg], inline=True)

In [ ]:
fig = histo(df_fast2[f'Cal_{P}_Bmag'], bins=50, inline=True)
fig = histo(df_fast2[f'Cal_{P}_Bmag_Avg'], bins=50, inline=True)
fig = histo(df_fast2_5[f'Cal_{P}_Bmag_Avg'], bins=50, inline=True)

In [ ]:
df_slow[" NMR [T]"].hist(bins=50)

In [ ]:
df_fast.eval("Cal_Res = (NMR - Cal_1_Bmag)", inplace=True)
df_fast.eval("Cal_Res_Rel = (NMR - Cal_1_Bmag) / NMR", inplace=True)

In [ ]:
fig = histo(df_fast2.Cal_4_Res_Rel, bins=50, inline=True)
fig = histo(df_fast2.Cal_4_Bmag_Avg_Res_Rel, bins=50, inline=True)
fig = histo(df_fast2_5.Cal_4_Bmag_Avg_Res_Rel, bins=50, inline=True)
#fig = histo(df_fast.Cal_1_Bmag_Avg_Res_Rel, bins=50, inline=True)

In [ ]:
fig = histo(df_slow[" NMR [T]"], bins=10, inline=True)

In [ ]:
df_slow[" NMR [T]"].plot()

In [ ]:
fig = scatter2d(df_slow.index, df_slow[" NMR [T]"], inline=True)

In [ ]:
fig = scatter2d(df_fast2.index, df_fast2[f"Cal_{P}_Bmag"], inline=True)

In [ ]:
import copy

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
P = 3
Ns = [1,5,10,30,60, 120,]# 240, 480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df[f"Cal_{P}_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])
[df[f"Cal_{P}_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N]

In [ ]:
df_fast_N[0][f"Cal_{P}_Bmag_Avg_Res_Rel"].describe()

In [ ]:
fig = scatter2d([df_fast2.index, df_fast_N[-2].index, df_slow.index], [df_fast2.Cal_4_Bmag,df_fast_N[-2].Cal_4_Bmag_Avg, df_slow[" NMR [T]"]], inline=True)

In [ ]:
fig = histo([df_fast.Cal_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in df_fast_N], bins=50,same_bins=True, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
 NMR [T]fig = histo([df_fast.Cal_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in [df_fast_N[0]]], bins=50,same_bins=False, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = histo([df.Cal_1_Bmag_Avg_Res_Rel for df in [df_fast_N[0]]], bins=50,same_bins=False, show_plot=True, opacity=0.5, inline=True)

In [ ]:
df_fast_N[0]["Cal_1_Bmag_Avg_Res_Rel"].describe()

In [ ]:
getsizeof(df_fast_N[0]) / 1000 / 1000

### Noise on all Hall probes

In [ ]:
#df_fast.columns

In [ ]:
'''for i in range(1,6):
    df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res = (NMR - Cal_{i}_Bmag)", inplace=True)
    df_fast.eval(f"Cal_{i}_Res_Rel = (NMR - Cal_{i}_Bmag) / NMR", inplace=True)

for i in range(1,6):
    for B in ["Bx","By","Bz"]:
        #df_fast.eval(f"Cal_{i}_Bmag = (Cal_{i}_Bx**2+Cal_{i}_By**2+Cal_{i}_Bz**2)**(1/2)", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res = (NMR - Cal_{i}_{B})", inplace=True)
        df_fast.eval(f"Cal_{i}_{B}_Res_Rel = (NMR - Cal_{i}_{B}) / NMR", inplace=True)

df_fast["seconds_delta"] = pd.to_timedelta(df_fast.index.values) / pd.offsets.Second(1) - 1565697533.416

df_slow["seconds_delta"] = pd.to_timedelta(df_slow.index.values) / pd.offsets.Second(1) - 1565697533.416''';

In [ ]:
#pd.to_timedelta(df_slow.index.values) / pd.offsets.Second(1)

In [ ]:
getsizeof(df_fast) / 1000 / 1000

In [ ]:
# relative residual

In [ ]:
for i in range(1,6):
    for B in ["Bx", "By", "Bz"]:
        fig = histo(df_fast[f"Cal_{i}_{B}_Res_Rel"], bins=50, show_plot=False, inline=True)
        filename = f"histo_Cal_{i}_{B}_Res_Rel"
        pio.write_image(fig, plotdir+f"relative_residuals/components/{filename}.pdf")
        pio.write_image(fig, plotdir+f"relative_residuals/components/{filename}.png")

In [ ]:
for i in range(1,6):
    fig = histo(df_fast[f"Cal_{i}_Res_Rel"], bins=50, show_plot=False, inline=True)
    filename = f"histo_Cal_{i}_Res_Rel"
    pio.write_image(fig, plotdir+f"relative_residuals/magnitude/{filename}.pdf")
    pio.write_image(fig, plotdir+f"relative_residuals/magnitude/{filename}.png")

In [ ]:
# residual

In [ ]:
for i in range(1,6):
    for B in ["Bx", "By", "Bz"]:
        fig = histo(df_fast[f"Cal_{i}_{B}_Res"], bins=50, show_plot=False, inline=True)
        filename = f"histo_Cal_{i}_{B}_Res"
        pio.write_image(fig, plotdir+f"residuals/components/{filename}.pdf")
        pio.write_image(fig, plotdir+f"residuals/components/{filename}.png")

In [ ]:
for i in range(1,6):
    fig = histo(df_fast[f"Cal_{i}_Res"], bins=50, show_plot=False, inline=True)
    filename = f"histo_Cal_{i}_Res"
    pio.write_image(fig, plotdir+f"residuals/magnitude/{filename}.pdf")
    pio.write_image(fig, plotdir+f"residuals/magnitude/{filename}.png")

In [ ]:
Ns = [5,10,30,60, 120, 240, 480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "Cal_1_Bmag_Avg"] = df_fast["Cal_1_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast["Cal_1_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))


### Check Field Angle and Average
- Leaving for historical purposes, but column names have changed

In [ ]:
'''df_fast["Cal_1_Transverse"] = (df_fast["Cal_1_Bx"]**2+df_fast["Cal_1_By"]**2)**(1/2)
df_fast["theta"] = np.arccos(df_fast["Cal_1_Bz"]/df_fast["Cal_1_Bmag"])
df_fast["phi"] = np.arccos(df_fast["Cal_1_Bx"]/df_fast["Cal_1_Transverse"])''';

In [ ]:
fig = histo(df_fast.theta*180/np.pi, bins=50, inline=True)
fig = histo(df_fast.phi*180/np.pi, bins=50, inline=True)

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [5,10,30,60, 120, 240, ]#480, 960, 1930,1930*2, 1930*4, 1930*8]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, "theta_Avg"] = df_fast["theta"].rolling(window = N, center=True).mean()
    df_fast.loc[:, "phi_Avg"] = df_fast["phi"].rolling(window = N, center=True).mean()
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_fast.theta]+[df.theta_Avg for df in df_fast_N], bins=50,same_bins=True, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
iplot(fig)

In [ ]:
fig = scatter2d(df_fast.index, df_fast.theta, inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast.phi, inline=True)

In [ ]:
df_fast_N[0].theta_Avg.plot()

In [ ]:
fig = scatter2d([df_fast.index, df_fast_N[2].index, df_slow.index], [df_fast.Cal_1_Bmag,df_fast_N[2].Cal_1_Bmag_Avg, df_slow[" NMR [T]"]], inline=True)

### Plots for Email

In [ ]:
len(df_fast)

Last time: 66240

In [ ]:
795048 / 60 / 60 /24

In [ ]:
66240 / 60 / 60

In [ ]:
# get probe name dictionary
directory = "/home/ckampa/Dropbox/LogFiles/"
filename_fast = '2019-09-04 080000.txt'

with open(directory+filename_fast, 'r') as file:
    lines = file.readlines()[6:9]

In [ ]:
lines

In [ ]:
id_reg = re.compile(": (.+?)  Offset")

In [ ]:
hall_ids_list = [id_reg.findall(line)[0] for line in lines]

In [ ]:
hall_ids_list

In [ ]:
hall_ids = {i+1: hall for i,hall in enumerate(hall_ids_list)}

In [ ]:
hall_ids

In [ ]:
N = 3

In [ ]:
len(df_fast)

In [ ]:
fig = scatter2d(df_fast[::100].index, df_fast[::100][f"Raw_{N}_Bx"], inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast[f"Raw_{N}_Bz"], inline=True)

In [ ]:
fig = scatter2d(df_fast.index, df_fast[f"Cal_{N}_Bz"], inline=True)

In [ ]:
fig = scatter2d(5*[df_fast.index], [df_fast[f'Cal_{i}_T'] for i in range(1,4)])

In [ ]:
fig = scatter2d([df_fast.seconds_delta, df_slow.seconds_delta], [df_fast[f'Cal_1_T'], df_slow[" Hall Element"]],show_plot=False)

In [ ]:
figprof = py_profile(df_fast.seconds_delta, df_fast.Cal_1_T, x_bins=50, show_plot=False)

In [ ]:
figprof2 = py_profile(df_slow.seconds_delta, df_slow[" Hall Element"], x_bins=50,xrange=[df_fast.seconds_delta.min(),df_fast.seconds_delta.max()], show_plot=False)

In [ ]:
fig.add_traces([figprof.data[0],figprof2.data[0]])

In [ ]:
df_fast.loc[:,"HallElement"] = [df_slow.iloc[df_slow.index.get_loc(row, method='nearest')][" Hall Element"] for row in df_fast.index]

In [ ]:
#fig = scatter2d(df_fast.HallElement, df_fast.Cal_1_T, markers=True, lines=False)

In [ ]:
fig = py_profile(df_fast.HallElement, df_fast.Cal_1_T, x_bins=12)

In [ ]:
df_fast["Cal_4_Bmag"].describe()

In [ ]:
fig = scatter2d(df_fast.index, df_fast["Cal_1_Bmag"], inline=True)

In [ ]:
fig = scatter2d(df_slow_old.index, df_slow_old[" NMR [T]"], inline=True)

In [ ]:
fig = scatter2d(df_slow.index, df_slow[" NMR [T]"], inline=True)

In [ ]:
start = 0
end = 600
x_bins = 20

In [ ]:
fig = scatter2d(df_fast.seconds_delta[start:end], df_fast.Cal_1_Bmag[start:end],show_plot=False)

In [ ]:
figprof = py_profile(df_fast.seconds_delta[start:end], df_fast.Cal_1_Bmag[start:end], x_bins = x_bins, show_plot=False)

In [ ]:
fig.add_trace(figprof.data[0])

In [ ]:
#fig.update_traces(visible=False)

In [ ]:
pio.write_image(fig, plotdir+"scatter_plus_profile_test.pdf")

In [ ]:
start = 1000
end = 1600
x_bins = 20

comp_old = "Mag"
comp_new = "Bmag"

In [ ]:
# old data
fig = scatter2d(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end],show_plot=False)
figprof = py_profile(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end], x_bins = x_bins, show_plot=False)
fig.add_trace(figprof.data[0])

In [ ]:
for i in range(1,6):
    fig = scatter2d(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end],show_plot=False)
    figprof = py_profile(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end], x_bins = x_bins, show_plot=False)
    fig.add_trace(figprof.data[0])
    iplot(fig)

In [ ]:
start = 1000
end = 1120
x_bins = 6

In [ ]:
# old data
fig = scatter2d(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end],show_plot=False)
figprof = py_profile(df_slow_old.seconds_delta[start:end], df_slow_old[f"Cal_{comp_old}"][start:end], x_bins = x_bins, show_plot=False)
fig.add_trace(figprof.data[0])

In [ ]:
for i in range(1,6):
    fig = scatter2d(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end],show_plot=False)
    figprof = py_profile(df_fast.seconds_delta[start:end], df_fast[f'Cal_{i}_{comp_new}'][start:end], x_bins = x_bins, show_plot=False)
    fig.add_trace(figprof.data[0])
    iplot(fig)

In [ ]:
fig = histo(df_fast.Cal_1_Res_Rel, bins=50, show_plot=False)
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
iplot(fig)

In [ ]:
df_slow_old["Cal_Res_Rel"] = (df_slow_old[" NMR [T]"] - df_slow_old["Cal"]) / df_slow_old[" NMR [T]"]

In [ ]:
df_slow_old = df_slow_old.loc["2019-08-10 12:30:00":]

In [ ]:
fig = histo(df_slow_old.Cal_Mag_Res_Rel, bins=50, show_plot=False)
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
iplot(fig)

In [ ]:
df_slow_old.columns

In [ ]:
## AVERAGING

In [ ]:
df_fast["day"] = df_fast.index.day

In [ ]:
import copy

Average of residuals

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
P = 3
Ns = [10,60,120, 300, 1000]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    #df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, f"Cal_{P}_Res_Rel_Avg"] = df_fast[f"Cal_{P}_Res_Rel"].rolling(window = N, center=True).mean()
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_fast[f'Cal_{P}_Res_Rel']]+[df[f'Cal_{P}_Res_Rel_Avg'] for df in df_fast_N], bins=100,
            same_bins=False, show_plot=False, opacity=0.5, inline=True, cut=[0.,0.999])
fig.update_traces(histnorm='probability') # "probability density"
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
#iplot(fig)

In [ ]:
pplot(fig)

Residuals of average

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
P = 3
Ns = [10,60,120, 300, 1000]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_fast.loc[:, f"Cal_{P}_Bmag_Avg"] = df_fast[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
    df_fast.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_fast["NMR"] - df_fast[f"Cal_{P}_Bmag_Avg"])/df_fast["NMR"]
    df_fast_N.append(copy.copy(df_fast[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_fast[f'Cal_{P}_Res_Rel']]+[df[f'Cal_{P}_Bmag_Avg_Res_Rel'] for df in df_fast_N], bins=100,
            same_bins=False, show_plot=False, opacity=0.5, inline=True, cut=[0.,0.999])
fig.update_traces(histnorm='probability') # "probability density"
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
#iplot(fig)

In [ ]:
pplot(fig)

In [ ]:
fig.layout.xaxis.title = "(NMR [T] - |Hall| [T]) / (NMR [T])"
fig.layout.title = "Histo: Relative Residuals<br>Averaging: NONE, 10 s, 60 s"

filename = plotdir+"rel_residuals_avg"
pio.write_image(fig, filename+".pdf")
pio.write_image(fig, filename+".png")

In [ ]:
fig = px.histogram(df_fast_N[1], x='Cal_3_Bmag_Avg_Res_Rel', histnorm="probability", color="day")
fig.show()

In [ ]:
fig = px.histogram(df_fast_N[1], x='Cal_3_Bmag_Avg_Res_Rel', histnorm="probability", color="day")
fig.show()

In [ ]:
#df_slim = df_fast.query("day in [5,6,7,8,9,10]")
df_slim = df_fast.query("5 <= day <= 10")

In [ ]:
len(df_slim)

In [ ]:
fig = px.histogram(df_slim, x='Cal_3_Bmag_Avg_Res_Rel', histnorm="probability density", color="day")
fig.show()

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
P = 3
Ns = [10,60,120, 300, 1000]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_slim.loc[:, f"Cal_{P}_Bmag_Avg"] = df_slim[f"Cal_{P}_Bmag"].rolling(window = N, center=True).mean()
    df_slim.loc[:, f"Cal_{P}_Bmag_Avg_Res_Rel"] = (df_slim["NMR"] - df_slim[f"Cal_{P}_Bmag_Avg"])/df_slim["NMR"]
    df_fast_N.append(copy.copy(df_slim[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_slim[f'Cal_{P}_Res_Rel']]+[df[f'Cal_{P}_Bmag_Avg_Res_Rel'] for df in df_fast_N], bins=100,
            same_bins=False, show_plot=False, opacity=0.5, inline=True, cut=[0.,0.999])
fig.update_traces(histnorm='probability') # "probability density"
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
iplot(fig)

In [ ]:
# Seeing effect of number of points in average
#N = 10#30#10#20#10#30#60
Ns = [10,60]#[5, 10, 30, 60]

df_fast_N = []

for N in Ns:
    df_slow_old.loc[:, "Cal_1_Bmag_Avg"] = df_slow_old["Cal_Mag"].rolling(window = N, center=True).mean()
    df_slow_old.loc[:, "Cal_1_Bmag_Avg_Res_Rel"] = (df_slow_old[" NMR [T]"] - df_slow_old["Cal_1_Bmag_Avg"])/df_slow_old[" NMR [T]"]
    df_fast_N.append(copy.copy(df_slow_old[N // 2::N]))
    #print(df_fast_N[-1]["Cal_1_Bmag_Avg_Res_Rel"].describe())

#df_fast_5 = df_fast[N // 2::N]
#print([df["Cal_1_Bmag_Avg_Res_Rel"].describe() for df in df_fast_N])

In [ ]:
fig = histo([df_slow_old.Cal_Mag_Res_Rel]+[df.Cal_1_Bmag_Avg_Res_Rel for df in df_fast_N], bins=15,same_bins=False, show_plot=False, opacity=0.5, inline=True)
fig.update_traces(histnorm='probability density')
fig.layout.xaxis.title = ("(NMR - |B_hall|) / NMR")
fig.layout.yaxis.title = ("Normalized Counts")
iplot(fig)